## Project: Implementing a ChatGPT App with LangChain from Scratch



In [10]:
%pip install --upgrade -q langchain 
%pip install --upgrade -q langchain-community
%pip install --upgrade -q openai 


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#       SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in German.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     verbose=True
# )

# if the code above gives an error use this:
chain = prompt | llm | StrOutputParser()

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit','q', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  hi, how are you?


Hallo! Mir geht es gut, danke. Wie kann ich Ihnen helfen?
--------------------------------------------------


Your prompt:  quit


Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [12]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  quit


Goodbye!


## Saving Chat Sessions

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Earth mass is ...


{'content': 'Earth mass is ...', 'chat_history': [], 'text': "The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."}
--------------------------------------------------


Your prompt:  what about the Moon and Sun?


{'content': 'what about the Moon and Sun?', 'chat_history': [HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth.")], 'text': "The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system."}
--------------------------------------------------


Your prompt:  quit


Goodbye!


In [4]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."), HumanMessage(content='what about the Moon and Sun?'), AIMessage(content="The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system.")]
